# Uppgift 3 - KPIer & explorativ dataanalys

In [267]:
import pandas as pd
from datetime import datetime # Jag vill kunna komma åt och plotta på ett smidigare sätt diverse tidserier sen.
import plotly_express as px


path = "Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx"
#vaccination_över_tid_regioner_raw = pd.read_excel(path, sheet_name = "Vaccinerade tidsserie")# repl.
vaccination_över_tid_raw = pd.read_excel(path, sheet_name = "Vaccinerade tidsserie")
vaccination_kön_raw = pd.read_excel(path, sheet_name = "Vaccinerade kön")
vaccination_ålder_raw = pd.read_excel(path, sheet_name = "Vaccinerade ålder")




## KPI 1 och 2. Vaccineringstatus över hela Svergie. Minst 1 dos, färdigvaccinerad på 2 dos.
Objektiv : Andel vaccinerade 1.00 för dos 1 och dos 2. Destö närmare 1 destö bättre 

In [277]:
vaccination_över_tid = vaccination_över_tid_raw[vaccination_över_tid_raw["Region"] == "| Sverige |"]
vaccination_över_tid["Tidspunkt"] = vaccination_över_tid["Vecka"].astype(str) + "-" + vaccination_över_tid["År"].astype(str) #Function it ?
vaccination_över_tid = vaccination_över_tid.drop(["Vecka", "År"], axis = 1)
vaccination_över_tid["Datum"] = vaccination_över_tid["Tidspunkt"].apply(lambda x : datetime.strptime(x + "-1","%W-%Y-%w"))

fig_1 = px.line(vaccination_över_tid, x = "Datum", y = "Andel vaccinerade", color = "Vaccinationsstatus")
fig_1.show()
fig_1.write_html("Data/html/andel_dos1_dos2.html")


/var/folders/f7/yr1v3cr12blbcs1mz2hr4b_m0000gn/T/ipykernel_35823/3544145212.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## KPI 3. Andel män vaccinerade.
Män vaccinerar sig mindre än kvinnor, vi vill kunna se den nationella performansen.
Objektiv är att minska skillnaden i andelen mellan vaccinerade män och kvinnor, skillnaden av andel är KPIn, och det är bra när den minskar.


In [278]:
vaccination_kön_raw = pd.read_excel(path, sheet_name = "Vaccinerade kön")
vaccination_kön = vaccination_kön_raw[vaccination_kön_raw["Kön"] != "Totalt"]

fig_2 = px.bar(vaccination_kön, x ="Vaccinationsstatus", y ="Andel vaccinerade", color = "Kön" , barmode='group')
fig_2.show()
fig_2.write_html("Data/html/andel_kön.html")



## KPI 4. Första dos i de tre minsta vaccinerade regionerna
- I uppgift 2 har vi sett att de tre största svenska regionerna hade också minst andel vaccinerade.
- Jag har kollat dos 1, som är entén till vaccineringsprocessen. Det är viktigt att övertyga ovaccinerade till första dosen. 
- KPIn är andel som har tagit 1 dos vaccin i regionerna Skåne, Stockholm och VG. Jag tycker att det är viktigts att titta på denna KPIn just för dessa regioner.

In [270]:
vaccination_över_tid_regioner = vaccination_över_tid_raw[vaccination_över_tid_raw["Region"] != "| Sverige |"] # Rensar dataframe
vaccination_över_tid_regioner = vaccination_över_tid_regioner[vaccination_över_tid_regioner["Vaccinationsstatus"] == "Minst 1 dos" ]

vaccination_över_tid_regioner["Tidspunkt"] = vaccination_över_tid_regioner["Vecka"].astype(str) + "-" + vaccination_över_tid_regioner["År"].astype(str)
vaccination_över_tid_regioner = vaccination_över_tid_regioner.drop(["Vecka", "År"], axis = 1)
vaccination_över_tid_regioner["Datum"] = vaccination_över_tid_regioner["Tidspunkt"].apply(lambda x : datetime.strptime(x + "-1","%W-%Y-%w"))  # Veckonummer och år räcker inte för att generera ett datum, däarför har jag behövt lägga "-1" som är måndagen i veckan.

vaccination_över_tid_regioner.head()

,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus,Tidspunkt,Datum
2,Stockholm,288,0.000148,Minst 1 dos,52-2020,2020-12-28
4,Uppsala,8,0.000025,Minst 1 dos,52-2020,2020-12-28
6,Södermanland,116,0.000474,Minst 1 dos,52-2020,2020-12-28
8,Östergötland,214,0.000554,Minst 1 dos,52-2020,2020-12-28
10,Jönköping,43,0.000144,Minst 1 dos,52-2020,2020-12-28


In [279]:
vaccination_över_tid_regioner_low_3 = vaccination_över_tid_regioner[vaccination_över_tid_regioner["Region"].isin(["Stockholm", "Västra Götaland", "Skåne"])] #På så sätt behöller jag de tre regionerna som jag har kunnat se hade lägst andel vaccinerade


fig_3 = px.line(vaccination_över_tid_regioner_low_3, x = "Tidspunkt", y = "Andel vaccinerade", color = "Region")
fig_3.show()
fig_3.write_html("Data/html/andel_storregionerna.html")

## 8 sista veckorna i datan:
- Dags att filtrera data med hjälp av datum.
- Jag vill "zooma" på slutet av kurvan.

Det är svårt att få de sista procenterna att ta första vaccindos i Sveriges tre största regioner.
Jag har försökt kolla på evenemang för att m"marknadsföra" vaccinationen under dessa veckor

Under vecka 36, september 11 , Region Skåne riktade sig mot en mindre vaccinerade grupp och erbjöd grattis vaccin på Malmö FF match.
Resultatet inte tillräckligt signifikativt för att påverka kurvans lutning.
Verkar vara en liten tendens upåt mellan v.40 och v.41 men har inte kunnat relaterva till ett särskilt händelse eller informationskampanj.

https://www.svt.se/nyheter/lokalt/skane/fotboll-och-en-spruta-vid-match

In [272]:
vaccination_över_tid_regioner_low_3_last_8_weeks = vaccination_över_tid_regioner_low_3.set_index('Datum')
vaccination_över_tid_regioner_low_3_last_8_weeks = vaccination_över_tid_regioner_low_3_last_8_weeks.loc["2021-08-23" : "2021-10-11"] # Med datetimes some index kan jag välja en loc.

In [280]:
fig_4 = px.line(vaccination_över_tid_regioner_low_3_last_8_weeks, x = "Tidspunkt", y = "Andel vaccinerade", color = "Region")
fig_4.show()
fig_4.write_html("Data/html/ande_storregionerna_sista_veckorna.html")


## KPI 5. Skillnaderna mellan 12-40 och 40 + grupparna
KPI är distansen mellan andel vaccinerade i grupperna 40 plus och yngre. Objektiv är att denna skillnad ska minska.
Jag har titta på alla regioner och klassificerat i vilka regioner som skillnaden mellan 40+ och yngre är störst 

In [274]:
vaccination_ålder = vaccination_ålder_raw[vaccination_ålder_raw['Region'] != "| Sverige |"] # Tar bort hela landet från regioner
vaccination_ålder = vaccination_ålder[vaccination_ålder['Åldersgrupp'] != "Totalt"] # Tar bort subtotals
vaccination_ålder = vaccination_ålder[vaccination_ålder['Vaccinationsstatus'] != "Färdigvaccinerade"] # tar bort färdigvaccinerade för att inte få dubbel count
vaccination_ålder["Population"] = vaccination_ålder["Antal vaccinerade"] / vaccination_ålder["Andel vaccinerade"] # Behöver återskappa en population för att återskappa en sub-andel för 40 plusare och 40-
vaccination_ålder["Age low"] = vaccination_ålder["Åldersgrupp"].str[:2].astype(float) # Behöver lägga en sån float kolum för att kunna filtrera gruper med logiska operator


vaccination_äldre = vaccination_ålder[vaccination_ålder["Age low"] > 40]
vaccination_yngre = vaccination_ålder[vaccination_ålder["Age low"] < 40]
vaccination_yngre = vaccination_yngre[vaccination_ålder["Age low"] > 12] # Behöver filtrera ytterligare för att få bort 12-15 klassen från data


vaccination_äldre_population = vaccination_äldre.groupby(["Region"])["Population"].sum() #Function it ?
vaccination_äldre_vaccinerade = vaccination_äldre.groupby(["Region"])["Antal vaccinerade"].sum()

accination_yngre_population = vaccination_yngre.groupby(["Region"])["Population"].sum()
vaccination_yngre_vaccinerade = vaccination_yngre.groupby(["Region"])["Antal vaccinerade"].sum()

vaccination_äldre_analys = pd.concat([vaccination_äldre_population, vaccination_äldre_vaccinerade], axis = 1)
vaccination_äldre_analys["Andel"] = vaccination_äldre_analys["Antal vaccinerade"] / vaccination_äldre_analys["Population"]
vaccination_äldre_analys["Klass"] = "40 plus"

vaccination_yngre_analys = pd.concat([accination_yngre_population, vaccination_yngre_vaccinerade], axis = 1)
vaccination_yngre_analys["Andel"] = vaccination_yngre_analys["Antal vaccinerade"] / vaccination_yngre_analys["Population"]
vaccination_yngre_analys["Klass"] = "under 40"

vaccination_analys = pd.concat([vaccination_yngre_analys, vaccination_äldre_analys])
vaccination_analys = vaccination_analys.reset_index()



/var/folders/f7/yr1v3cr12blbcs1mz2hr4b_m0000gn/T/ipykernel_35823/56290219.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [281]:
fig_5 = px.bar(vaccination_analys, x ="Region", y ="Andel", color = "Klass" , barmode='group')
fig_5.show()
fig_5.write_html("Data/html/andel_aldgrupp_region.html")

In [282]:
vaccination_further_analys = pd.merge(vaccination_yngre_analys, vaccination_äldre_analys, left_index = True, right_index= True) # Jag har valt en sån merge här fär att kunna jämföra andelarna på region
vaccination_further_analys["Andel skillnad"] = vaccination_further_analys["Andel_y"] - vaccination_further_analys["Andel_x"]
vaccination_further_analys = vaccination_further_analys.reset_index()

fig_6 = px.bar(vaccination_further_analys, x ="Region", y ="Andel skillnad")
fig_6.update_layout(xaxis={'categoryorder':'total descending'})
fig_6.show()
fig_6.write_html("Data/html/andel_aldgrupp_skillnad.html")